# Agentes en LangGraph con Gemini


## Caso 1 LLM estructurado

<img src="https://langchain-ai.github.io/langgraph/tutorials/workflows/img/augmented_llm.png" alt="gate" width="600"/>

Este primer ejemplo muestra cómo utilizar LangGraph y el modelo Gemini de Google para crear un agente conversacional que genere consultas de búsqueda optimizadas y justificaciones usando salidas estructuradas con Pydantic.

Necesitarás una API key de Google AI Studio para usar Gemini. Puedes obtenerla en: https://makersuite.google.com/app/apikey

In [1]:
import os
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI

# Declaración y llamado de la key del modelo LLM
# use esta función para definir la key del modelo LLM si tuvo problemas con el archivo .env.json o create_env.py
def _set_env(var: str):
   if not os.environ.get(var):
       os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

### Paso 1: Definir un esquema de salida estructurada con Pydantic
Usamos Pydantic para definir una clase que representa la estructura de la respuesta que queremos obtener del modelo de lenguaje.
En este caso, la clase `SearchQuery` tiene dos campos: `search_query` (la consulta optimizada para búsqueda web) y `justification` (la justificación de por qué esa consulta es relevante para la petición del usuario).


In [2]:
from pydantic import BaseModel, Field

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Consulta optimizada para búsqueda web.")
    justification: str = Field(
        None, description="Por qué esta consulta es relevante para la petición del usuario."
    )

### Paso 2: Configurar el modelo LLM para que devuelva una salida estructurada
Aquí, usamos el método `with_structured_output` para indicarle al modelo que queremos que su respuesta siga el esquema definido por la clase `SearchQuery`.


In [3]:
structured_llm = llm.with_structured_output(SearchQuery)

### Paso 3: Invocar el modelo con una pregunta y obtener la respuesta estructurada
Ahora, hacemos una pregunta al modelo y recibimos la respuesta siguiendo el esquema definido anteriormente.

In [4]:
output = structured_llm.invoke("¿Cómo se relaciona la puntuación de calcio en la tomografía computarizada con el colesterol alto?")
output

SearchQuery(search_query='relación entre la puntuación de calcio de la TC y el colesterol alto', justification='El usuario pregunta por la relación entre la puntuación de calcio de la TC y el colesterol alto.')

### Paso 4: Definir una herramienta personalizada
Ahora vamos a definimos una función llamada `multiply` que multiplica dos números. Esta función se puede usar como una herramienta que el modelo puede invocar cuando lo considere necesario.


In [18]:
def multiply(a: float, b: float) -> float:
    """
    Multiplica dos números float y devuelve el resultado. 
    Ejemplo: multiplicación de a=4.0 por b=2.0 retorna 8.0

    Args:
        a (float): Primer número a multiplicar.
        b (float): Segundo número a multiplicar.

    Returns:
        float: El resultado de multiplicar a por b.
    """
    result = a*b
    return(result)

### Paso 5: Integrar la herramienta al modelo LLM
Usamos el método `bind_tools` para agregar la función `multiply` como una herramienta disponible para el modelo.

In [19]:
llm_with_tools = llm.bind_tools([multiply])


### Paso 6: Invocar el modelo para que utilice la herramienta
Hacemos una pregunta que requiere el uso de la herramienta de multiplicación. El modelo detecta la intención y llama a la función `multiply`.


In [21]:
msg = llm_with_tools.invoke("Cúanto es la multiplicación de 5.0 por 3.0?")
print(msg)

content='' additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"b": 3.0, "a": 5.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []} id='run--d3e84e47-5396-4fd4-826e-87489bd849d5-0' tool_calls=[{'name': 'multiply', 'args': {'b': 3.0, 'a': 5.0}, 'id': '78903932-8f35-42f2-8561-929aaeb840f5', 'type': 'tool_call'}] usage_metadata={'input_tokens': 71, 'output_tokens': 5, 'total_tokens': 76, 'input_token_details': {'cache_read': 0}}


### Paso 7: Obtener la llamada a la herramienta realizada por el modelo
Finalmente, podemos ver los detalles de la llamada a la herramienta que realizó el modelo con `msg.tool_calls`. Incluso podemos acceder a los argumentos que ha logrado extraer el LLM y ejecutar manualmente la función


In [16]:
# Acceder a los argumentos de la tool
msg.tool_calls

[{'name': 'multiply',
  'args': {'b': 3.0, 'a': 5.0},
  'id': '01df873b-8fbe-431a-95b3-ae5c3184b275',
  'type': 'tool_call'}]

In [17]:

# Si tu mensaje es 'msg'
if msg.tool_calls:
    for tool_call in msg.tool_calls:
        if tool_call['name'] == 'multiply':
            # Ejecutar la función manualmente
            a= tool_call['args']['a']
            b= tool_call['args']['b']
            print ("argumento a: ",a, "argumento b: ", b)
            print ("resultado: ", multiply(a,b))

argumento a:  5.0 argumento b:  3.0
resultado:  15.0
